In [ ]:
import os
#os.environ['LOGS_DIRECTORY'] = '../eval_logs'
os.environ['LOGS_DIRECTORY'] = './logs'

import sys
sys.path.insert(0, '..')

In [ ]:
import json
import random

from tqdm.auto import tqdm
from pydantic import BaseModel
from pydantic_ai import Agent

from main import initialize_index, initialize_agent
from logs import log_interaction_to_file, LOG_DIR

In [ ]:
LOG_DIR.absolute()

In [ ]:
index = initialize_index()
agent = initialize_agent(index)

In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()
# print(os.environ.get("GROQ_API_KEY"))

question_generation_prompt = """
You are helping to create test questions for an AI agent that answers questions about a data engineering course.

Based on the provided FAQ content, generate realistic questions that students might ask.

The questions should:

- Be natural and varied in style
- Range from simple to complex
- Include both specific technical questions and general course questions

Generate one question for each record.
""".strip()

class QuestionsList(BaseModel):
    questions: list[str]

question_generator = Agent(
    name="question_generator",
    instructions=question_generation_prompt,
    model='groq:llama-3.1-8b-instant'
    output_type=QuestionsList
)

In [ ]:
sample = random.sample(index.docs, 10)
prompt = [d['content'] for d in sample]

In [ ]:
result = await question_generator.run(user_prompt=json.dumps(prompt))
question_list = result.output

In [ ]:
for question in tqdm(question_list.questions):
    print(question)
    result = await agent.run(user_prompt=question)
    print(result.output)
    log_interaction_to_file(agent, result.new_messages(), source='ai-generated')
    print()